In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [4]:
'''the tfds.load method loads the data set. if the dataset is missing then, its first 
downloaded''' 

mnist_dataset, mnist_info=tfds.load(name="mnist", with_info=True, as_supervised=True)

In [10]:
#the mnist_train and mnist_test datasets have the training and the testing data
mnist_train, mnist_test=mnist_dataset['train'], mnist_dataset['test']
#since the split only converts the entire data into test and train, we can divide the train data further to validation data
num_validation_samples=0.1 * mnist_info.splits['train'].num_examples
'''tf.cast(variable, datatype): converts a variable into the data type'''

num_validation_samples=tf.cast(num_validation_samples, tf.int64)

def scale(image, label):
    image=tf.cast(image, tf.float32)
    image/=255.
    return image, label

scaled_train_and_validation_data=mnist_train.map(scale)
test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000
shuffled_train_and_validation_data=scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data=shuffled_train_and_validation_data.take(num_validation_samples)
train_data=shuffled_train_and_validation_data.skip(num_validation_samples)


BATCH_SIZE=100

train_data=train_data.batch(BATCH_SIZE)
validation_data=validation_data.batch(num_validation_samples)
num_test_samples=mnist_info.splits['test'].num_examples
test_data=test_data.batch(num_test_samples)

validation_inputs, validation_targets=next(iter(validation_data))

In [20]:
input_size=784
output_size=10
hidden_layer_size=100

model=tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')
                          ])

In [21]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [22]:
NUM_EPOCHS=5
model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 2s - loss: 0.3352 - accuracy: 0.9034 - val_loss: 0.1648 - val_accuracy: 0.9513
Epoch 2/5
540/540 - 2s - loss: 0.1320 - accuracy: 0.9610 - val_loss: 0.1169 - val_accuracy: 0.9657
Epoch 3/5
540/540 - 2s - loss: 0.0947 - accuracy: 0.9714 - val_loss: 0.0870 - val_accuracy: 0.9738
Epoch 4/5
540/540 - 2s - loss: 0.0745 - accuracy: 0.9774 - val_loss: 0.0692 - val_accuracy: 0.9805
Epoch 5/5
540/540 - 2s - loss: 0.0572 - accuracy: 0.9826 - val_loss: 0.0676 - val_accuracy: 0.9807


In [24]:
test_loss, test_accuracy=model.evaluate(test_data)
print("Test loss= {0:.2f}. Test accuracy= {1:.2f}%".format(test_loss, test_accuracy*100.))

1/1 [==============================] - 0s 191ms/step - loss: 0.0822 - accuracy: 0.9750
Test loss= 0.08. Test accuracy= 97.50%
